In [ ]:
import numpy as np # linear algebra
import pandas as pd
import zipfile
import time
import torch
import torchvision as tv
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torchvision.datasets.vision import StandardTransform
from torchvision.utils import make_grid
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import sys
%matplotlib inline

DATA_DIR = "./data"

In [ ]:
#Instantiating CUDA device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
with zipfile.ZipFile("facial-keypoints-detection.zip", 'r') as zip_ref:
    zip_ref.extractall("data")

In [ ]:
training_data = pd.read_csv(f'{DATA_DIR}/training.zip')
training_data = training_data.dropna()

In [ ]:
X = [np.fromstring(image, dtype=np.uint8, sep=' ').reshape(96,96) for image in training_data["Image"]]
X = np.reshape(X,(-1,96, 96))
Y = training_data[["left_eye_center_x","left_eye_center_y"]].values.reshape(-1,2).astype('float32')
X.shape, Y.shape

In [ ]:
from matplotlib import pyplot as plt
for i in range(5):
    plt.imshow(X[i], interpolation='nearest',cmap="gray")
    plt.plot(Y[i][0],Y[i][1],marker=".",color="red")
    plt.show()

In [ ]:
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt

preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

X2 = []
for i in range(len(X)):
    img = Image.fromarray(X[i]).convert("RGB")
    img = preprocess(img)
    X2.append(np.array(img))

In [ ]:
Y2 = Y * (224/96)

In [ ]:
for i in range(5):
    plt.imshow(X2[i].transpose(1,2,0), interpolation='nearest')
    plt.plot(Y2[i][0],Y2[i][1],marker=".",color="red")
    plt.show()

In [ ]:
X_Training = np.reshape(X2,(-1, 3, 224, 224))
X_Training.shape

In [ ]:
import torchvision as tv
import torch.nn as nn

net = tv.models.resnet18(pretrained=True)
net.fc = nn.Linear(512,2)

#Output final architecture
net.eval()

In [ ]:
#Define Loss
loss_function = nn.MSELoss()

#Define Optimizer(SGD)
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

#worker-cores
if sys.platform.startswith('win'):
    num_workers = 0
else:
    num_workers = 4

#batch-size
batch_size = 40

In [ ]:
trainloader = torch.utils.data.DataLoader(list(zip(X_Training,Y2)), batch_size=batch_size, shuffle=True, num_workers=num_workers)

#Move the input and model to GPU for speed if available
net.to(device)

# Specify the number of epochs for training
num_epochs = 2
#Instatiate Logs
running_loss_history = []
running_corrects_history = []

#Train the net
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    running_corrects = 0.0
    net.train()

    for inputs, labels in trainloader:

        # forward + backward + optimize
        output = net(inputs)
        loss = loss_function(output,labels)
        running_loss += loss

        # setting initial parameter gradients to 0
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        # print statistics
        print("loss:",loss.item())
        #running_corrects += torch.sum(preds == labels.data) # calculating te accuracy by taking the sum of all the correct predictions in a batch.

    epoch_loss = running_loss/len(trainloader) # loss per epoch
    #epoch_acc = running_corrects.float()/ len(trainloader) # accuracy per epoch
    running_loss_history.append(epoch_loss) # appending for displaying
    #running_corrects_history.append(epoch_acc)
    # net.eval()

    print('Epoch:', epoch)
    print('training loss: {:.4f}'.format(epoch_loss))